In [1]:
# --- Imports and Setup ---
import numpy as np
import pandas as pd
import gc
import os
import joblib

# PyTorch and Dataloader
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Scikit-learn and UMAP
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
import umap

# AnnData for loading
import anndata as ad
from scipy.sparse import issparse




# --- Global Configuration ---
name = 'cite'

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# --- Helper Functions ---

def neg_correlation_score_torch(y_pred: torch.Tensor, y_true: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Row-wise Pearson correlation for PyTorch Tensors.
    """
    x = y_true - y_true.mean(dim=1, keepdim=True)
    y = y_pred - y_pred.mean(dim=1, keepdim=True)
    num = (x * y).sum(dim=1)
    den = torch.sqrt((x.square().sum(dim=1) + eps) * (y.square().sum(dim=1) + eps))
    r = num / den
    return -r.mean()

def pearson_corr_numpy(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    """
    Calculates mean row-wise Pearson correlation for NumPy arrays.
    """
    # Calculate row-wise correlation
    corrs = np.array([np.corrcoef(p, t)[0, 1] for p, t in zip(y_pred, y_true)])
    # Return the mean of correlations, handling potential NaNs
    return np.nanmean(corrs)

from scipy.sparse import issparse
import warnings

def clr_transform(X):
    """
    Applies a robust Centered Log-Ratio (CLR) transformation.
    This version handles NaNs, infs, and negative values in the input.
    The output will be a dense matrix.
    """
    print("  Applying robust CLR transformation...")
    
    if issparse(X):
        num_samples = X.shape[0]
        num_features = X.shape[1]
        X_clr = np.zeros((num_samples, num_features), dtype=np.float32)
        chunk_size = 1000 # Process in chunks to save memory
        
        for i in range(0, num_samples, chunk_size):
            chunk = X[i:i+chunk_size].toarray()

            # 1. Sanitize any existing NaNs/Infs to 0
            np.nan_to_num(chunk, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
            # 2. Clip any negative values to 0
            chunk[chunk < 0] = 0
            # 3. Add pseudocount
            chunk += 1
            
            # Apply log transformation and centering
            log_chunk = np.log(chunk)
            X_clr[i:i+chunk_size] = log_chunk - log_chunk.mean(axis=1, keepdims=True)
            
    else: # If input is already dense
        # Create a copy to avoid modifying the original array outside the function scope
        X_dense = X.copy()

        # 1. Sanitize any existing NaNs/Infs to 0
        np.nan_to_num(X_dense, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
        # 2. Clip any negative values to 0
        X_dense[X_dense < 0] = 0
        # 3. Add pseudocount
        X_dense += 1
        
        # Apply log transformation and centering
        log_X = np.log(X_dense)
        X_clr = log_X - log_X.mean(axis=1, keepdims=True)
        
    return X_clr

Using device: cpu


In [2]:
# --- Cell 2: Data Loading and Preprocessing (SVD->UMAP for X, SVD-only for y) ---
import joblib
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

# --- Dimensional Hyperparameters ---
SVD_COMPONENTS_X = 256 # Intermediate SVD components for input X
SVD_COMPONENTS_Y = 128  # Intermediate SVD components for target y
UMAP_COMPONENTS_X = 128       # Final UMAP dimension for X
UMAP_N_NEIGHBORS = 15
UMAP_MIN_DIST = 0.1

print("Loading sparse data from .h5ad files...")
PATH_TRAIN_INP = f"train_{name}_inputs.h5ad"
PATH_TRAIN_TGT = f"train_{name}_targets.h5ad"
PATH_TEST_INP  = f"test_{name}_inputs.h5ad"

adata_train_inp = ad.read_h5ad(PATH_TRAIN_INP)
adata_train_tgt = ad.read_h5ad(PATH_TRAIN_TGT)
adata_test_inp = ad.read_h5ad(PATH_TEST_INP)

train_X_sparse = adata_train_inp.X
test_X_sparse = adata_test_inp.X
train_y_sparse = adata_train_tgt.X

print(f"Initial sparse train_X shape: {train_X_sparse.shape}")
print(f"Initial sparse train_y shape: {train_y_sparse.shape}")

# --- Input Matrix Preprocessing (train_X, test_X) ---
print("\n--- Preprocessing Input Matrices (X) with CLR -> SVD -> UMAP... ---")
# Step 1: CLR transformation
train_X_clr = clr_transform(train_X_sparse)
test_X_clr = clr_transform(test_X_sparse)

# Step 2: SVD Pre-reduction
print(f"Step 2: Performing TruncatedSVD on X to {SVD_COMPONENTS_X} components...")
svd_x = TruncatedSVD(n_components=SVD_COMPONENTS_X, random_state=42)
train_X_svd = svd_x.fit_transform(train_X_clr)
test_X_svd = svd_x.transform(test_X_clr)

# Step 3: UMAP decomposition
print(f"Step 3: Performing UMAP on SVD-reduced X to {UMAP_COMPONENTS_X} components...")
mapper_x = umap.UMAP(n_components=UMAP_COMPONENTS_X, n_neighbors=UMAP_N_NEIGHBORS,
                   min_dist=UMAP_MIN_DIST, random_state=42)
train_X_umap = mapper_x.fit_transform(train_X_svd)
test_X_umap = mapper_x.transform(test_X_svd)
print(f"Finished X preprocessing. Final train_X_umap shape: {train_X_umap.shape}")


# --- Target Matrix Preprocessing (train_y) ---
print("\n--- Preprocessing Target Matrix (y) with CLR -> SVD... ---")
# Step 1: CLR transformation
train_y_clr = clr_transform(train_y_sparse) # Keep this for validation metric

# Step 2: SVD-only reduction for a robust inverse transform
print(f"Step 2: Performing TruncatedSVD on y to {SVD_COMPONENTS_Y} components...")
svd_y = TruncatedSVD(n_components=SVD_COMPONENTS_Y, random_state=42)
train_y_svd = svd_y.fit_transform(train_y_clr)
print(f"Finished y preprocessing. Final train_y_svd shape: {train_y_svd.shape}")

# Save ONLY the svd_y object
joblib.dump(svd_y, 'svd_y_model.joblib')
print("Saved svd_y model.")

# --- Clean up unused variables from memory ---
print("\nCleaning up memory...")
# ... (cleanup code remains the same, but variables have changed)
del adata_train_inp, adata_train_tgt, adata_test_inp
del train_X_sparse, test_X_sparse, train_y_sparse
del train_X_clr, test_X_clr, train_X_svd, test_X_svd
# keep train_y_clr for validation
gc.collect()

# --- Final Check ---
print("\nPreprocessing complete. Final shapes for training:")
print(f"train_X_umap: {train_X_umap.shape}")
print(f"train_y_svd: {train_y_svd.shape}")
print(f"test_X_umap: {test_X_umap.shape}")

Loading sparse data from .h5ad files...
Initial sparse train_X shape: (64074, 15435)
Initial sparse train_y shape: (64074, 140)

--- Preprocessing Input Matrices (X) with CLR -> SVD -> UMAP... ---
  Applying robust CLR transformation...
  Applying robust CLR transformation...
Step 2: Performing TruncatedSVD on X to 256 components...
Step 3: Performing UMAP on SVD-reduced X to 128 components...


c:\Users\21138\anaconda3\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Finished X preprocessing. Final train_X_umap shape: (64074, 128)

--- Preprocessing Target Matrix (y) with CLR -> SVD... ---
  Applying robust CLR transformation...
Step 2: Performing TruncatedSVD on y to 128 components...
Finished y preprocessing. Final train_y_svd shape: (64074, 128)
Saved svd_y model.

Cleaning up memory...

Preprocessing complete. Final shapes for training:
train_X_umap: (64074, 128)
train_y_svd: (64074, 128)
test_X_umap: (48663, 128)


In [3]:
# --- Cell 3: Dataset Class ---
class MultiTargetDataset(Dataset):
    def __init__(self, features, targets_low_dim, targets_full_dim=None):
        """
        Custom Dataset that returns low-dim features, low-dim targets (for loss),
        and full-dim targets (for validation metric).
        """
        self.features = features # This will be X_umap
        self.targets_low_dim = targets_low_dim # This will be y_svd
        self.targets_full_dim = targets_full_dim # This will be y_clr
        self.is_eval = targets_full_dim is not None

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        
        if self.is_eval:
            target_low = torch.tensor(self.targets_low_dim[idx], dtype=torch.float32)
            target_full = torch.tensor(self.targets_full_dim[idx], dtype=torch.float32)
            return feature, target_low, target_full
        else: # For test set
            return feature

In [4]:
# --- Cell 4 : Model Architecture ---
class FNNModel(nn.Module):
    def __init__(self, num_features, target_dim, hidden_size=1024, dropout=0.25):
        """
        A simple Feedforward Neural Network (FNN/MLP).
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.GELU(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.Tanh(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_size // 2, target_dim)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# --- Cell 5: Loss function ---
class CombinedLoss(nn.Module):
    """
    Combines MSE loss and negative Pearson correlation loss in the latent space.
    """
    def __init__(self):
        super().__init__()
        self.mse_loss = nn.MSELoss()
        self.corr_loss = neg_correlation_score_torch

    def forward(self, pred_low_dim, true_low_dim):
        # Calculate losses in the UMAP latent space
        loss_mse = self.mse_loss(pred_low_dim, true_low_dim)
        loss_corr = self.corr_loss(pred_low_dim, true_low_dim)
        return loss_mse + loss_corr

In [6]:
#--- Cell 6: Training function ---
def train_one_fold(
    train_loader,
    valid_loader,
    model,
    optimizer,
    loss_fn,
    svd_y,
    train_params,
    n_fold,
    model_name
):
    """
    Trains the FNN model for a single fold.
    Validation metric is Pearson correlation on the original (CLR-transformed) space.
    """
    best_val_corr = -1.0
    model_path = f"model_{model_name}_fold_{n_fold+1}.pth"

    for epoch in range(train_params['epochs']):
        # --- Training Phase ---
        model.train()
        for features, targets_low, _ in train_loader:

            features, targets_low = features.to(DEVICE), targets_low.to(DEVICE)
            optimizer.zero_grad()
            preds_low = model(features)
            loss = loss_fn(preds_low, targets_low)
            loss.backward()
            optimizer.step()

        # --- Validation Phase ---
        model.eval()
        val_loss_latent = 0.0
        all_recon_preds = []
        all_full_targets = []
        
        with torch.no_grad():
            for features, targets_low, targets_full in valid_loader:
                features, targets_low = features.to(DEVICE), targets_low.to(DEVICE)
                
                # Predict in the latent space
                preds_low = model(features)
                
                # Calculate loss in the latent (SVD) space
                loss = loss_fn(preds_low, targets_low)
                val_loss_latent += loss.item() * len(features)
                
                # Inverse transform predictions to the full (CLR) space for correlation metric
                recon_preds = svd_y.inverse_transform(preds_low.cpu().numpy())
                all_recon_preds.append(recon_preds)
                all_full_targets.append(targets_full.numpy())
        
        # Calculate and report metrics for the epoch
        val_loss_latent /= len(valid_loader.dataset)
        val_preds_full = np.concatenate(all_recon_preds)
        val_targets_full = np.concatenate(all_full_targets)
        val_corr = pearson_corr_numpy(val_preds_full, val_targets_full)
        
        # Save the model if the Pearson correlation on the validation set has improved
        if val_corr > best_val_corr:
            best_val_corr = val_corr
            torch.save(model.state_dict(), model_path)
            
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch+1}/{train_params['epochs']}, "
                  f"Latent Loss: {val_loss_latent:.4f}, "
                  f"Validation Pearson Corr: {val_corr:.4f}")
    
    print(f"Best validation Pearson correlation for fold {n_fold+1}: {best_val_corr:.4f}")
    return model_path

In [7]:
#--- Cell 7: Train and Validation ---
MODEL_PARAMS = {
    'num_features': train_X_umap.shape[1], 
    'target_dim': train_y_svd.shape[1], # Model's target is the SVD-reduced y
    'hidden_size': 1024,
    'dropout': 0.25,
}
TRAIN_PARAMS = {'batch_size': 512, 'epochs': 50, 'lr': 1e-3}
folds = KFold(n_splits=5, shuffle=True, random_state=42)

# --- Initialize Prediction Arrays ---
sub_preds = np.zeros((test_X_umap.shape[0], train_y_clr.shape[1]), dtype=np.float32)

svd_y_loaded = joblib.load('svd_y_model.joblib')

# --- Main CV Loop ---
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_X_umap)):
    print(f"\n===== Processing Fold {n_fold+1}/{folds.get_n_splits()} =====")
    
    # 1. Slice all necessary data arrays for the current fold
    X_train_fold, X_valid_fold = train_X_umap[train_idx], train_X_umap[valid_idx]
    y_train_fold, y_valid_fold = train_y_svd[train_idx], train_y_svd[valid_idx]
    y_clr_train_fold, y_clr_valid_fold = train_y_clr[train_idx], train_y_clr[valid_idx]
    
    # 2. Prepare DataLoaders, Model, Optimizer, and Loss
    train_dataset = MultiTargetDataset(X_train_fold, y_train_fold, y_clr_train_fold)
    valid_dataset = MultiTargetDataset(X_valid_fold, y_valid_fold, y_clr_valid_fold)
    train_loader = DataLoader(train_dataset, batch_size=TRAIN_PARAMS['batch_size'], shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=TRAIN_PARAMS['batch_size'] * 2, num_workers=0)
    
    model = FNNModel(**MODEL_PARAMS).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=TRAIN_PARAMS['lr'])
    loss_fn = CombinedLoss().to(DEVICE)

    # 3. Call the training function
    best_model_path = train_one_fold(
        train_loader, valid_loader, model, optimizer, loss_fn,
        svd_y_loaded, # Pass the SVD mapper for inverse transform in validation
        TRAIN_PARAMS, n_fold, f"{name}_hybrid_fnn"
    )
    
    # 4. Load best model and perform predictions on the test set
    model.load_state_dict(torch.load(best_model_path))
    model.eval()
    
    # Create DataLoader for the test set
    test_dataset = MultiTargetDataset(test_X_umap, None, None) # No targets for the test set
    test_loader = DataLoader(test_dataset, batch_size=TRAIN_PARAMS['batch_size'] * 2)
    
    test_preds_list_low_dim = []
    with torch.no_grad():
        for features in test_loader:
            # Predict in the low-dimensional space
            preds_low_dim = model(features.to(DEVICE))
            test_preds_list_low_dim.append(preds_low_dim.cpu().numpy())
            
    test_preds_low_dim = np.concatenate(test_preds_list_low_dim)
    
    # Inverse transform predictions from SVD space back to CLR space
    test_preds_clr = svd_y_loaded.inverse_transform(test_preds_low_dim)
    
    # Accumulate predictions for each fold
    sub_preds += test_preds_clr / folds.get_n_splits()
            
    # Clean up memory for the next fold
    del model, optimizer, loss_fn, train_dataset, valid_dataset, train_loader, valid_loader
    gc.collect()
    torch.cuda.empty_cache()

# --- Save final results ---
np.save(f'sub_preds_{name}.npy', sub_preds)
print(f'\nTest predictions saved, shape: {sub_preds.shape}')


===== Processing Fold 1/5 =====
  Epoch 10/50, Latent Loss: -0.5474, Validation Pearson Corr: 0.7927
  Epoch 20/50, Latent Loss: -0.6040, Validation Pearson Corr: 0.8078
  Epoch 30/50, Latent Loss: -0.6373, Validation Pearson Corr: 0.8248
  Epoch 40/50, Latent Loss: -0.6199, Validation Pearson Corr: 0.8162
  Epoch 50/50, Latent Loss: -0.6326, Validation Pearson Corr: 0.8224
Best validation Pearson correlation for fold 1: 0.8266

===== Processing Fold 2/5 =====
  Epoch 10/50, Latent Loss: -0.5673, Validation Pearson Corr: 0.8044
  Epoch 20/50, Latent Loss: -0.6080, Validation Pearson Corr: 0.8121
  Epoch 30/50, Latent Loss: -0.6225, Validation Pearson Corr: 0.8169
  Epoch 40/50, Latent Loss: -0.6140, Validation Pearson Corr: 0.8141
  Epoch 50/50, Latent Loss: -0.5620, Validation Pearson Corr: 0.7867
Best validation Pearson correlation for fold 2: 0.8272

===== Processing Fold 3/5 =====
  Epoch 10/50, Latent Loss: -0.5407, Validation Pearson Corr: 0.7890
  Epoch 20/50, Latent Loss: -0.5

In [8]:
# --- Cell 8: Cleanup ---

# Final step: remove all saved model files
print("\nCleaning up saved model files...")
for n_fold in range(folds.get_n_splits()):
    model_path = f"model_{name}_hybrid_fnn_fold_{n_fold+1}.pth"
    try:
        os.remove(model_path)
        print(f"Removed: {model_path}")
    except OSError as e:
        print(f"Error removing {model_path}: {e}")

# Also remove the SVD mapper file
try:
    os.remove('svd_y_model.joblib')
    print("Removed: svd_y_model.joblib")
except OSError as e:
    print(f"Error removing svd_y_model.joblib: {e}")

print("\nCleanup complete. Only prediction files remain.")


Cleaning up saved model files...
Removed: model_cite_hybrid_fnn_fold_1.pth
Removed: model_cite_hybrid_fnn_fold_2.pth
Removed: model_cite_hybrid_fnn_fold_3.pth
Removed: model_cite_hybrid_fnn_fold_4.pth
Removed: model_cite_hybrid_fnn_fold_5.pth
Removed: svd_y_model.joblib

Cleanup complete. Only prediction files remain.
